In [1]:
from pymongo import MongoClient
from bson.objectid import ObjectId

In [2]:
client = MongoClient('act4dgem.cse.iitd.ac.in', serverSelectionTimeoutMS=600000)
db = client['eventwise_media-db']

In [3]:
resolved_entities_1 = db.get_collection('all_media_entities_resolved_final')

In [4]:
resolved_entities_2 = db.get_collection('all_media_entities_resolved_final3')

In [5]:
with open('./relationships_works_in.txt') as fil:
    lines = fil.readlines()
instances = [l[10:].strip().split('\t')[:2] for l in lines if l.startswith('instance')]
sentences = [l[10:].strip() for l in lines if l.startswith('sentence')]

In [8]:
name = []
name_uuid = []
org = []
org_uuid = []
sen = []
for i,inst in enumerate(instances):
    res = resolved_entities_1.find_one({'_id':ObjectId(inst[0]), 'graphid':{'$exists':True}, 'type':'Person'})
    s = sentences[i]
    if res:
        name.append(res['stdName'])
        name_uuid.append(res['graphid'])
        s = s.replace(inst[0], res['stdName'])
    else:
        continue
    res = resolved_entities_2.find_one({'_id':ObjectId(inst[1]), 'graphid':{'$exists':True}})
    if res:
        org.append(res['stdName'])
        org_uuid.append(res['graphid'])
        s = s.replace(inst[1], res['stdName'])
    else:
        del name[-1]
        continue
    sen.append(s)

In [9]:
len(name) == len(org)

True

In [11]:
import csv

In [16]:
with open('relatedInstancesWorksIn.csv', 'w') as fil:
    writer = csv.writer(fil)
    for i in range(len(name)):
        writer.writerow([name[i], name_uuid[i], org[i], org_uuid[i], sen[i]])

In [17]:
len(name)

294

In [18]:
len(sen)

294

In [19]:
len(org)

294

In [20]:
with open('relatedInstancesInGraphDb_WorksIn.txt', 'w') as fil:
    for i in range(len(sen)):
        print(name[i], '\t' , org[i], '\n', sen[i], '\n', '-'*50, file=fil)